In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
abhikjha_imdb_wiki_faces_dataset_path = kagglehub.dataset_download('abhikjha/imdb-wiki-faces-dataset')

print('Data source import complete.')

100%|██████████| 5.92G/5.92G [01:08<00:00, 92.5MB/s]

Extracting files...


Data source import complete.


In [ ]:
!find / -type d -name "*imdb-wiki-faces-dataset*" 2>/dev/null

/root/.cache/kagglehub/datasets/abhikjha/imdb-wiki-faces-dataset


In [ ]:
!cp -r /root/.cache/kagglehub/datasets/abhikjha/imdb-wiki-faces-dataset /content/

In [12]:
!pip install -q tensorflow opencv-python
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [26]:
train_df = pd.read_csv('/content/imdb_train.csv')
valid_df = pd.read_csv('/content/imdb_valid.csv')
test_df  = pd.read_csv('/content/imdb_test.csv')

# Preview
print("Train:", len(train_df), "Val:", len(valid_df), "Test:", len(test_df))
train_df.head()

Train: 183887 Val: 45972 Test: 56087


,filename,age,gender,x_min,y_min,x_max,y_max,head_roll,head_yaw,head_pitch
0,88/nm0810488_rm2655369984_1947-5-16_2010.jpg,63,M,437.303192,42.856476,467.050140,98.899902,-5.962659,-36.795117,1.780944
1,45/nm0339745_rm995732224_1978-5-26_1986.jpg,8,M,364.144257,786.160950,552.663513,1009.277405,5.570054,6.371778,4.237167
2,55/nm0420955_rm3863263232_1947-5-4_2008.jpg,61,M,1137.936035,242.395020,1331.409424,549.516113,7.377303,2.632854,17.795181
3,26/nm0000126_rm489339392_1955-1-18_2007.jpg,52,M,441.024567,56.378716,563.802246,237.178558,3.875414,-21.536889,16.404904
4,23/nm0000623_rm1856542720_1954-2-17_2005.jpg,51,F,68.119370,53.637310,133.536194,142.539185,7.606505,21.275758,1.785451


In [27]:
BASE_PATH = "/content/imdb-wiki-faces-dataset/versions/2/imdb_crop"
for df in [train_df, valid_df, test_df]:
    df['img_path'] = df['filename'].apply(lambda x: os.path.join(BASE_PATH, x))
    df['age'] = df['age'].astype(np.float32)

In [28]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [29]:
class AgeDataGenerator(Sequence):
    def __init__(self, df, batch_size=32, img_size=(224, 224), shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def __getitem__(self, index):
        batch_df = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = []
        batch_ages = []

        for _, row in batch_df.iterrows():
            img = load_img(row['img_path'], target_size=self.img_size)
            img = img_to_array(img) / 255.0  # Normalize
            batch_images.append(img)
            batch_ages.append(row['age'])

        return np.array(batch_images), np.array(batch_ages)

In [30]:
train_gen = AgeDataGenerator(train_df, batch_size=32)
valid_gen = AgeDataGenerator(valid_df, batch_size=32)
test_gen  = AgeDataGenerator(test_df, batch_size=32, shuffle=False)

In [31]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='linear')(x)

In [32]:
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mae', metrics=['mae'])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 224, 224,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_2     │ (None, 224, 224,  │          7 │ rescaling_4[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_5         │ (None, 224, 224,  │          0 │ normalization_2[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_5[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 4,171,645 (15.91 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [33]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# 🔸 Path to save best model
checkpoint_path = 'best_age_model.keras'

# 🔹 Callbacks
early_stop = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True, verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1
)

model_ckpt = ModelCheckpoint(
    checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1
)

# 🔸 Start Training
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=50,  # can change as needed
    callbacks=[early_stop, reduce_lr, model_ckpt],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 10.7049 - mae: 10.7049
Epoch 1: val_loss improved from inf to 8.47145, saving model to best_age_model.keras
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 998s 158ms/step - loss: 10.7046 - mae: 10.7046 - val_loss: 8.4715 - val_mae: 8.4715 - learning_rate: 1.0000e-04
Epoch 2/50
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 7.9192 - mae: 7.9192
Epoch 2: val_loss did not improve from 8.47145
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 837s 146ms/step - loss: 7.9191 - mae: 7.9191 - val_loss: 8.8948 - val_mae: 8.8948 - learning_rate: 1.0000e-04
Epoch 3/50
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 7.5394 - mae: 7.5394
Epoch 3: val_loss improved from 8.47145 to 8.03799, saving model to best_age_model.keras
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 851s 148ms/step - loss: 7.5394 - mae: 7.5394 - val_loss: 8.0380 - val_mae: 8.0380 - learning_rate: 1.0000e-04
Epoch 4/50
5747/5747 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 7.2771 - mae: 7.2771
Epoch 4: val

KeyboardInterrupt: 

In [35]:
from google.colab import files
files.download("best_age_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>